# Homework 2
## Exercise 1: Spam Filter

I will walk you step by step through the spam filtering process. 

First, we define a function which returns a dictionary which contains a count for every word occurrence in the provided email list

In [1]:
def get_hash_table(email_list):
    hash_table_result = {}
    for email in email_list:
        for word in email:
            if word in hash_table_result.keys():
                print("Found another ", word)
                hash_table_result[word] = hash_table_result[word] + 1.0
            else:
                print("Creating entry for ", word)
                hash_table_result[word] = 1.0
    return hash_table_result

Here are the hash tables that generate for the example data set

In [2]:
spam_corpus = [["I", "am", "spam", "spam", "I", "am"], ["I", "do", "not", "like", "that", "spamiam"]]
ham_corpus = [["do", "i", "like", "green", "eggs", "and", "ham"], ["i", "do"]]

print("Hash Tables")
ham_hash_table = get_hash_table(ham_corpus)
spam_hash_table = get_hash_table(spam_corpus)
print(ham_hash_table)
print(spam_hash_table)

Hash Tables
Creating entry for  do
Creating entry for  i
Creating entry for  like
Creating entry for  green
Creating entry for  eggs
Creating entry for  and
Creating entry for  ham
Found another  i
Found another  do
Creating entry for  I
Creating entry for  am
Creating entry for  spam
Found another  spam
Found another  I
Found another  am
Found another  I
Creating entry for  do
Creating entry for  not
Creating entry for  like
Creating entry for  that
Creating entry for  spamiam
{'do': 2.0, 'i': 2.0, 'like': 1.0, 'green': 1.0, 'eggs': 1.0, 'and': 1.0, 'ham': 1.0}
{'I': 3.0, 'am': 2.0, 'spam': 2.0, 'do': 1.0, 'not': 1.0, 'like': 1.0, 'that': 1.0, 'spamiam': 1.0}


Then we need to finish gathering information about the email lists before we generate the probability table

In [3]:
num_ham_emails = len(ham_corpus)
num_spam_emails = len(spam_corpus)
print("Num Ham Emails:", num_ham_emails)
print("Num Spam Emails:", num_spam_emails)

Num Ham Emails: 2
Num Spam Emails: 2


Finally we create the probability table from the algorithm provided by Graham.

In [4]:
def get_probability_value(word, ham_hash_table, spam_hash_table, num_ham_emails, num_spam_emails):
    result = 0.0
    g_value = 0.0
    b_value = 0.0
    if word in ham_hash_table.keys():
        g_value = 2.0 * ham_hash_table[word]
    if word in spam_hash_table.keys():
        b_value = spam_hash_table[word]
    if (g_value + b_value > 1.0):
        result = max(0.01, min(0.99, min(1.0, b_value/num_spam_emails) / (min(1.0, g_value/num_ham_emails) + min(1.0, b_value/num_spam_emails))))
    return result


def get_probability_table(num_ham_emails, ham_hash_table, num_spam_emails, spam_hash_table):
    result_probability_table = {}
    # add good words to table
    for good_word in ham_hash_table:
        result_probability_table[good_word] = get_probability_value(good_word, ham_hash_table, spam_hash_table, num_ham_emails, num_spam_emails)
    # add remaining bad words to table
    for bad_word in spam_hash_table:
        if bad_word in result_probability_table.keys():
            # Do nothing
            print()
        else:
            result_probability_table[bad_word] = get_probability_value(bad_word, ham_hash_table, spam_hash_table, num_ham_emails, num_spam_emails)
    return result_probability_table

Here is the probability table

In [5]:
probability_table = get_probability_table(num_ham_emails, ham_hash_table, num_spam_emails, spam_hash_table)
print("Probability Table\n", probability_table)



Probability Table
 {'do': 0.3333333333333333, 'i': 0.01, 'like': 0.3333333333333333, 'green': 0.01, 'eggs': 0.01, 'and': 0.01, 'ham': 0.01, 'I': 0.99, 'am': 0.99, 'spam': 0.99, 'not': 0.0, 'that': 0.0, 'spamiam': 0.0}


Now we can create a probability filter which uses the probability table

In [6]:
def get_probability_of_spam(probability_table, email):
    prod = 1.0
    complement = 1.0
    for word in email:
        prod = prod * probability_table[word]
        complement = complement * (1.0 - probability_table[word])

    return prod / (prod + complement)

Lets print the probabilities for each of the emails that we started with

In [7]:
print(get_probability_of_spam(probability_table, ["I", "am", "spam", "spam", "I", "am"]))
print(get_probability_of_spam(probability_table, ["I", "do", "not", "like", "that", "spamiam"]))
print(get_probability_of_spam(probability_table, ["do", "i", "like", "green", "eggs", "and", "ham"]))
print(get_probability_of_spam(probability_table, ["i", "do"]))

0.9999999999989378
0.0
2.6288392819642677e-11
0.005025125628140704


## Response 

Graham argues that this is a Baysian approach to SPAM. What makes it Bayesian?. This program could be considered bayesian because it implements statistics.

## Exercise 2

#### Note: I had to do this exercise twice as my jupyter notebook didnt save which was why was turned in an hour after midnight Thursday.

### a. Implement the network using the AIMA Python tools.

In [8]:
from probability import BayesNet, enumeration_ask, elimination_ask, gibbs_ask

# Utility variables
T, F = True, False

wetnet = BayesNet([
    ('Cloudy', '', 0.5),
    ('Sprinkler', 'Cloudy', {T: 0.1, F: 0.5}),
    ('Rain', 'Cloudy', {T: 0.8, F: 0.2}),
    ('WetGrass', 'Sprinkler Rain', {(T, T): 0.99, (T, F): 0.9, (F, T): 0.9, (F, F): 0.0}),
    ])

### b. Compute the number of independent values in the full joint probability distribution for this domain. Assume that no conditional independence relations are known to hold between these values.
Since there are 4 variables each of which can have a true or false, there are 16 possible independent values in the full joint probability distribution for this domain.


### c. Compute the number of independent values in the Bayesian network for this domain. Assume the conditional independence relations implied by the Bayes network.
We can just look at Figure 14.12(a) and count the number of probabilities provided. The answer is 9.

### d. Compute probabilities for the following:

P(Cloudy) is given in Figure 14.12(a) as 0.5

In [9]:
print(enumeration_ask('Cloudy', dict(), wetnet).show_approx())

False: 0.5, True: 0.5


P(Sprinkler | cloudy) also look at Figure 14.12(a) = 0.1

In [10]:
print(enumeration_ask('Sprinkler', dict(Cloudy = T), wetnet).show_approx())

False: 0.9, True: 0.1


P(Cloudy| Sprinkler ^ ¬Rain) = 𝝰 P(Cloudy, Sprinkler, ¬Rain) = 𝝰 P(Cloudy) * P(Sprinkler | Cloudy) * P(¬Rain | Cloudy) = 𝝰 < 0.5 * 0.1 * 0.1, 0.5 * 0.5 * 0.8 > = 𝝰 <0.005, 0.2> = <0.0476, 0.952>

In [11]:
print(enumeration_ask('Cloudy', dict(Sprinkler = T, Rain = F), wetnet).show_approx())

False: 0.952, True: 0.0476


P(WetGrass | Cloudy ^ Sprinkler ^ Rain) = 𝝰 P(WetGrass, Cloudy, Sprinkler, Rain) = 𝝰 P(WetGrass | Sprinkler ^ Rain) * P(Sprinkler | Cloudy) * P(Rain | Cloudy) * P(Cloudy)
= 𝝰 < 0.99 * 0.1 * 0.8 * 0.5, 0.01 * 0.1 * 0.8 * 0.5>
= 𝝰 < 0.0396, 0.0004 > 
= < 0.99, 0.01 >

In [12]:
print(enumeration_ask('WetGrass', dict(Cloudy = T, Sprinkler = T, Rain = T), wetnet).show_approx())

False: 0.01, True: 0.99


P(Cloudy | ¬WetGrass) = 𝝰 P(¬WetGrass | Sprinkler ^ Rain) * P(Sprinkler | Cloudy) * P(Rain | Cloudy) * P(Cloudy)
= 𝝰 < 0.01 * 0.1 * 0.8 * 0.5, 0.99 * 0.1 * 0.8 * 0.5>
= < 0.639, 0.361 >

In [13]:
print(enumeration_ask('Cloudy', dict(WetGrass = F), wetnet).show_approx())

False: 0.639, True: 0.361
